In this brief workflow, we extract a flood water mask and then convert it into a shapefile using gdal.

Define environment variables and create gbdx interface.

In [ ]:
import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

Execute workflow.

In [6]:
# location of acomped multispectral image
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/flood-water/north-carolina/10400100238BDE00/ms'

# specify input to flood mapper
fm = gbdx.Task('flood-mapper')
fm.inputs.image = input_location

# vectorize output of flood mapper with gdal
vectorize = gbdx.Task('gdal-cli')
vectorize.inputs.data = fm.outputs.image.value
vectorize.inputs.execution_strategy = 'runonce'
vectorize.inputs.command = """gdal_polygonize.py -f "ESRI Shapefile" $indir/*.tif $outdir/vectors.shp"""
## this is for geojson format
#vectorize.inputs.command = """gdal_polygonize.py -f "GeoJSON" $indir/*.tif $outdir/vectors.geojson"""

wf = gbdx.Workflow([fm, vectorize])
wf.savedata(fm.outputs.image, 'platform-stories/trial-runs/flood-water/mask')
wf.savedata(vectorize.outputs.data, 'platform-stories/trial-runs/flood-water/shapefile')
wf.execute()

u'4722465786451634940'

In [7]:
wf.status

{u'event': u'started', u'state': u'running'}